In [4]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Q3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.cores.max", 2)\
        .getOrCreate()

spark_context = spark_session.sparkContext
        #.config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.4.0-3.3")\
spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/14 18:28:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
redComm1 = spark_session.read.json("hdfs://192.168.2.97:50000/user/ubuntu/RC_2008-03.json")
redComm2 = spark_session.read.json("hdfs://192.168.2.97:50000/user/ubuntu/RC_2008-04.json")

In [6]:
print(redComm1)

DataFrame[body: string, subreddit: string]


In [13]:
import requests

url = "http://files.pushshift.io/reddit/comments/RC_2006-01.zst"
filename = "RC_2006-01.zst"

response = requests.get(url)

with open(filename, "wb") as f:
    f.write(response.content)

In [14]:
import zstandard
import json

In [22]:
pip install zstandard

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 12.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
filename = "RC_2006-01.zst"

# Open the compressed file
with open(filename, "rb") as f:
    dctx = zstandard.ZstdDecompressor()
    with dctx.stream_reader(f) as reader:
        # Read the JSON data and parse it
        data = json.loads(reader.read().decode())

# Print the attributes of the first comment in the dataset
print(data[0].keys())

ZstdError: zstd decompress error: Frame requires too much memory for decoding

In [12]:
subCount = redComm1.select('subreddit')\
    .groupBy("subreddit")\
    .count()\
    .orderBy("count", ascending=False)

In [13]:
subCount.show()

+-------------+------+
|    subreddit| count|
+-------------+------+
|   reddit.com|207092|
|     politics| 83375|
|  programming| 34642|
|         pics| 25352|
|      science| 22874|
|     business| 14555|
|    worldnews| 12725|
|        funny| 10223|
|entertainment|  9403|
|       gaming|  9313|
|         nsfw|  4036|
|          WTF|  3276|
|      offbeat|  2709|
|   technology|  2645|
|    Economics|  2313|
|        obama|  1863|
|      gadgets|  1714|
|  environment|  1323|
|    AskReddit|  1133|
|       comics|  1029|
+-------------+------+
only showing top 20 rows



In [14]:
cons = redComm1[redComm1.subreddit == 'politics']

In [15]:
cons.show()

+--------------------+---------+
|                body|subreddit|
+--------------------+---------+
|Gore would be a p...| politics|
|           [deleted]| politics|
|Thanks for the ex...| politics|
|      who's design? | politics|
|&gt;&gt;I defende...| politics|
|Well, if Pelosi d...| politics|
|What will public ...| politics|
|&gt; trying to hi...| politics|
|Uh, what about th...| politics|
|It's staunchly an...| politics|
|That was kind of ...| politics|
|Good point. This ...| politics|
|pn6, anyone can l...| politics|
|Didn't seem that ...| politics|
|&gt; Hey there sf...| politics|
|           [deleted]| politics|
|that kind of thin...| politics|
|This occured only...| politics|
|When is someone I...| politics|
|No.  They both st...| politics|
+--------------------+---------+
only showing top 20 rows



In [20]:
cons.describe()

DataFrame[summary: string, body: string, subreddit: string]

In [16]:
output = cons.take(1)

In [17]:
first_comment = output[0].body

In [21]:
num_politics_comments = len(first_comment)

print("Number of comments in the politics selected subreddit: ", num_politics_comments)

Number of comments in the politics selected subreddit:  113


In [22]:
redComm1.describe()

DataFrame[summary: string, body: string, subreddit: string]